<style>
    .jp-CodeCell {
        overflow-x: auto;
    }
    .jp-CodeCell pre {
        white-space: pre-wrap !important;
        word-break: break-all !important;
    }
</style>

In [ ]:
import random
import math
import numpy as np
from matplotlib import pyplot as plt

#Creating a Pattern Set

In [ ]:
def generate_pattern_point(label=None):
    if label == None:
        r = math.sqrt(random.random())*2
        theta = random.random()*math.pi+math.pi/2
        if random.random() >= 0.5:
            theta += math.pi
            x1 = r*math.cos(theta)
            x2 = r*math.sin(theta) - 1
        else:
            x1 = r*math.cos(theta)
            x2 = r*math.sin(theta)
#        x1 = -2*random.random()
#        x2 = random.random()*math.sqrt((2-x1)*(2+x1))*2-math.sqrt((2-x1)*(2+x1))
        return np.array([x1, x2])
    else:
        print("problem")

In [ ]:
def labelize_pattern_point(x1, x2):
    if x1 < 0:
        if x1*x1+x2*x2 > 1.0:
            return "C1"
        elif x1*x1+x2*x2 < 1.0:
            return "C2"
        else:
            return "C1" if random.random() < 0.5 else "C2"
    elif x1 > 0:
        if x1*x1+(x2+1)*(x2+1) > 1.0:
            return "C2"
        elif x1*x1+(x2+1)*(x2+1) < 1.0:
            return "C1"
        else:
            return "C1" if random.random() < 0.5 else "C2"
    else:
        if x2 < -2 or (x2 < 1 and x2 > 0):
            return "C2"
        elif x2 > 1 or (x2 > -2 and x2 < -1):
            return "C1"
        else:
            return "C1" if random.random() < 0.5 else "C2"


In [ ]:
def generate_pattern_points(n, label=None):
    if label == None:
        return np.array([generate_pattern_point(None) for i in range(n)])

In [ ]:
def labelize_pattern_points(pattern_point_list):
    return np.array([labelize_pattern_point(x1, x2) for (x1, x2) in pattern_point_list])

In [ ]:
def plot_pattern_points(pattern_point_list, label_list):
    points_c1 = [pattern_point_list[i] for i in range(len(pattern_point_list)) if label_list[i] == 'C1']
    points_c2 = [pattern_point_list[i] for i in range(len(pattern_point_list)) if label_list[i] == 'C2']
    x_c1, y_c1 = zip(*points_c1)
    x_c2, y_c2 = zip(*points_c2)

    plt.figure(figsize=(6, 6))
    plt.scatter(x_c1, y_c1, color='red', s=1, label='C1')
    plt.scatter(x_c2, y_c2, color='blue', s=1, label='C2')
    plt.title(str(len(pattern_point_list))+' Pattern Points')
    plt.xlabel('x1')
    plt.ylabel('x2')
    plt.grid(True)
    plt.xlim(-3, 3)
    plt.ylim(-3, 3)
    plt.legend()
    plt.show()

In [ ]:
X = generate_pattern_points(150000, None)
y_label = labelize_pattern_points(X)
X = X.T
y_label = y_label.reshape(1, len(y_label))
print(X.shape)
print(y_label.shape)


In [ ]:
plot_pattern_points(X.T, y_label[0])

In [ ]:
#Not used for binary classification
def labels_to_one_hot(y_labels, classes):
    n_samples = len(y_labels)
    n_classes = len(classes)
    y_one_hot = np.zeros((n_samples, n_classes))
    for i, label in enumerate(y_labels):
        index = classes.index(label)
        y_one_hot[i, index] = 1
    return y_one_hot

In [ ]:
#Only for binary classification
def labels_to_float(y_labels, positive_class):
    return np.array([1 if label == positive_class else 0 for label in y_labels[0]])

In [ ]:
X_train = X[:, :int(len(X[0])*0.7)]
X_test = X[:, int(len(X[0])*0.7):]
y = np.array([labels_to_float(y_label, 'C1')])
y_train = y[:, :int(y.shape[1]*0.7)]
y_test = y[:, int(y.shape[1]*0.7):]
print(X_train)

In [ ]:
def normalize_pattern_points(X, X_train):
    mean = np.mean(X_train, axis=1).reshape(1, -1).T    
    std_dev = np.std(X_train, axis=1).reshape(1, -1).T
    X = (X - mean) / std_dev
    return X

In [ ]:
X_train = normalize_pattern_points(X_train, X_train)
X_test = normalize_pattern_points(X_test, X_test)

#Building the neural network

In [ ]:
def sigmoid(z):
    z = np.clip(z, -709, 709)
#    z = np.clip(z, -1000, 1000)
    return 1 / (1 + np.exp(-z))

In [ ]:
def relu(x):
    return np.maximum(x, 0)

In [ ]:
def softmax(z):
    e_z = np.exp(z - np.max(z))  # Subtract max for numerical stability
    return e_z / np.sum(e_z)

In [ ]:
def sigmoid_derivative(z):
    A = sigmoid(z)
    return A * (1 - A)

In [ ]:
def relu_derivative(z):
    grad = np.zeros(z.shape)
    grad[z > 0] = 1
    return grad

In [ ]:
#DOES NOT WORK TODO
def softmax_derivative(z):
    s = softmax(z)
    return np.diagflat(s) - np.outer(s, s)

In [ ]:
Z_test = np.array([-1, 0, 1])
print("Sigmoid derivative:", sigmoid_derivative(Z_test))
print("ReLU derivative:", relu_derivative(Z_test))

In [ ]:
def init_weights_normal_distribution(layers_dim, mean, std_dev):
    n = len(layers_dim)

    b_list = [0]*(n)
    w_list = [0]*(n)

    for i in range(1, n):
        w_list[i] = np.random.randn(layers_dim[i], layers_dim[i-1]) * std_dev + mean
        b_list[i] = np.zeros((layers_dim[i], 1))
    return w_list, b_list

In [ ]:
w_list, b_list = init_weights_normal_distribution([2, 2, 1], 0, 0.1)
print("w_list = {}".format(w_list))
print("b_list = {}".format(b_list))

In [ ]:
def forward(X, w_list, b_list):
    n_layers = len(w_list)
    A = X
    z_list = [0]*(n_layers)
    a_list = [X]*(n_layers)  # Initialize with input layer

    for i in range(1, n_layers):
        Z = np.dot(w_list[i], A) + b_list[i]
        A = relu(Z) if i < n_layers - 1 else sigmoid(Z)
        z_list[i] = Z
        a_list[i] = A
    return z_list, a_list, A

In [ ]:
X1 = np.array([1., 1.]).reshape(2, 1)
print("X = {}".format(X1))
z_list, a_list, A = forward(X1, w_list, b_list)
print(z_list)
print(a_list)
print(A)

In [ ]:
def cross_entropy_loss(A, y, epsilon=1e-8):
    assert A.shape == y.shape
    m = A.shape[1]
    loss = -1/m * (np.dot(y, np.log(A.T + epsilon)) + np.dot(1-y, np.log((1 - A).T + epsilon)))
    return np.squeeze(loss)

In [ ]:
A = np.array([[0.9, 0.3]])
y1 = np.array([[1, 0]])

loss = cross_entropy_loss(A, y1)
print(loss)

In [ ]:
def backward(w_list, b_list, z_list, a_list, X, y, epsilon=1e-8):
    n_layers = len(w_list)
    m = y.shape[0]
    dw_list = [0]*n_layers
    db_list = [0]*n_layers
    a_list[0] = X

    for i in range(n_layers-1, 0, -1):
        A, A_prev, Z = a_list[i], a_list[i-1], z_list[i]
        W = w_list[i]
        if i == n_layers-1:
            dA = -np.divide(y, A + epsilon) + np.divide(1 - y, 1 - A + epsilon)
        if i == n_layers-1:
            dZ = np.multiply(dA, sigmoid_derivative(Z))
        else:
            dZ = np.multiply(dA, relu_derivative(Z))
        dW = np.dot(dZ, A_prev.T)/m
        db = np.sum(dZ, axis=1, keepdims=True)/m

        dA = np.dot(W.T, dZ)

        dw_list[i] = dW
        db_list[i] = db

    return dw_list, db_list


In [ ]:
dw_list, db_list = backward(w_list, b_list, z_list, a_list, np.array([[1], [1]]), np.array([[1]]))
print(dw_list)
print(db_list)

In [ ]:
def optimize(w_list, b_list, dw_list, db_list, learning_rate):
    n_layers = len(w_list)
    for i in range(n_layers):
        if dw_list[i] is not None and db_list[i] is not None:
            w_list[i] -= learning_rate * dw_list[i] #TODO: adaptative learning rate
            b_list[i] -= learning_rate * db_list[i]
    return w_list, b_list


In [ ]:
def optimize_with_adam(w_list, b_list, dw_list, db_list, learning_rate, t, m_w_list, v_w_list, m_b_list, v_b_list, beta1=0.9, beta2=0.999, epsilon=1e-8):
    n_layers = len(w_list)
    for i in range(n_layers):
        if dw_list[i] is not None and db_list[i] is not None:
            # Update biased first moment estimate for weights and biases
            m_w_list[i] = beta1 * m_w_list[i] + (1 - beta1) * dw_list[i]
            m_b_list[i] = beta1 * m_b_list[i] + (1 - beta1) * db_list[i]

            # Update biased second raw moment estimate for weights and biases
            v_w_list[i] = beta2 * v_w_list[i] + (1 - beta2) * (dw_list[i] ** 2)
            v_b_list[i] = beta2 * v_b_list[i] + (1 - beta2) * (db_list[i] ** 2)

            # Compute bias-corrected first moment estimate for weights and biases
            m_w_hat = m_w_list[i] / (1 - beta1 ** t)
            m_b_hat = m_b_list[i] / (1 - beta1 ** t)

            # Compute bias-corrected second raw moment estimate for weights and biases
            v_w_hat = v_w_list[i] / (1 - beta2 ** t)
            v_b_hat = v_b_list[i] / (1 - beta2 ** t)

            # Update weights and biases
            w_list[i] -= learning_rate * m_w_hat / (np.sqrt(v_w_hat) + epsilon)
            b_list[i] -= learning_rate * m_b_hat / (np.sqrt(v_b_hat) + epsilon)

    return w_list, b_list, m_w_list, v_w_list, m_b_list, v_b_list

In [ ]:
def generate_batch(X, batch_size):
    n = X.shape[1]
    batches = [range(i, min(X.shape[1], i+batch_size)) for i in range(0, n, batch_size)]
    return batches

In [ ]:
def accuracy(y, y_pred):
    assert y.shape[0] == 1
    assert y.shape == y_pred.shape
    y_pred = np.round(y_pred)
    return float(np.dot(y, y_pred.T) + np.dot(1-y, 1-y_pred.T)) / y.size

In [ ]:
print(X_train.shape[1])

In [ ]:
# stop conditions: max_epochs and acc_stop
def train(X_train, y_train, layers, batch_size=100, max_epochs=100, learning_rate=0.1, acc_stop=0.99, loss_diff=0.01, validation_split=0.3, mean=0, std_dev=0.01, optimizer='Adam', verbose=True):
    X_validation = X_train[:, :int(validation_split*len(X_train[0]))]
    X_train = X_train[:, int(validation_split*len(X_train[0])):]
    y_validation = y_train[:, :int(validation_split*len(y_train[0]))]
    y_train = y_train[:, int(validation_split*len(y_train[0])):]

    train_loss_list = [0]*max_epochs
    validation_loss_list = [0]*max_epochs
    train_accuracy_list = []
    validation_accuracy_list = [0]*max_epochs
    
    m_w_list, v_w_list, m_b_list, v_b_list = [0]*len(layers), [0]*len(layers), [0]*len(layers), [0]*len(layers)
    # prepare batch training
    batches = generate_batch(X_train, batch_size)
    # init weights
    w_list, b_list = init_weights_normal_distribution(layers, mean, std_dev)

    epoch = 1
    while epoch <= max_epochs:
        avg_loss = 0
        
        for batch in batches:
            X = X_train[:, batch]
            y = y_train[:, batch]
            z_list, a_list, A = forward(X, w_list, b_list)
            dw_list, db_list = backward(w_list, b_list, z_list, a_list, X, y)
            if optimizer == 'Adam':
                w_list, b_list, m_w_list, v_w_list, m_b_list, v_b_list = optimize_with_adam(w_list, b_list, dw_list, db_list, learning_rate, epoch, m_w_list, v_w_list, m_b_list, v_b_list, beta1=0.9, beta2=0.999, epsilon=1e-10) #beta1=0.9 beta0.999
            else:
                w_list, b_list = optimize(w_list, b_list, dw_list, db_list, learning_rate)

            avg_loss += cross_entropy_loss(A, y)

        #Train loss
        train_loss_list[epoch-1] = avg_loss / len(batches) #loss calculation

        #Validation loss
        _, _, valid_A = forward(X_validation, w_list, b_list)
        validation_loss_list[epoch-1] = cross_entropy_loss(valid_A, y_validation)

        #Train accuracy
        _, _, A = forward(X_train, w_list, b_list)
        train_accuracy = accuracy(y_train, A)
        train_accuracy_list.append(train_accuracy)

        #Validation accuracy
        _, _, A = forward(X_validation, w_list, b_list)
        validation_accuracy_list[epoch-1] = accuracy(y_validation, A)

        if verbose:
            print(f"Epoch {epoch}/{max_epochs}, Loss: {train_loss_list[epoch-1]}, Accuracy: {train_accuracy}", end='\r')

        if validation_accuracy_list[epoch-1] > acc_stop and abs(validation_loss_list[epoch-1] - train_loss_list[epoch-1]) < loss_diff:
            break
        epoch += 1

    epoch = min(epoch, max_epochs)
    return w_list, b_list, epoch, train_loss_list[:epoch], validation_loss_list[:epoch], train_accuracy_list, validation_accuracy_list[:epoch]

In [ ]:
max_epochs = 50
w_list, b_list, nb_epochs, train_loss_list, validation_loss_list, train_accuracy_list, validation_accuracy_list = train(X_train, y_train, [2, 50, 25, 1], batch_size=500, max_epochs=max_epochs, learning_rate=0.005, acc_stop = 0.999, loss_diff=0.002, validation_split = 0.2, mean=0, std_dev=0.05, optimizer='Adam')

In [ ]:
max_epochs = 50
w_list, b_list, nb_epochs, train_loss_list, validation_loss_list, train_accuracy_list, validation_accuracy_list = train(X_train, y_train, [2, 50, 25, 1], batch_size=1500000, max_epochs=max_epochs, learning_rate=0.001, acc_stop = 0.99, loss_diff=0.002, validation_split = 0.2, mean=0, std_dev=0.05)

In [ ]:
max_epochs = 100
w_list, b_list, nb_epochs, train_loss_list, validation_loss_list, train_accuracy_list, validation_accuracy_list = train(X_train, y_train, [2, 50, 25, 1], 500, max_epochs, learning_rate=0.01, acc_stop = 0.99, loss_diff=0.002, validation_split = 0.2, mean=0, std_dev=0.05)

#Analysis and Evaluation

In [ ]:
max_epochs = 50
n_avg = 10
#hyperparameters = [1, 10, 100, 200, 500, 750, 900, 1000, 1100, 1200, 1300, 1400, 1500, 2000, 2500, 5000, 10000, len(X_train[0])] #batch size
#hyperparameters = [5e-6, 1e-5, 5e-5, 0.0001, 0.00025, 0.0005, 0.001, 0.0025, 0.005, 0.01, 0.05] #learning rate
hyperparameters = [0.00025, 0.0005, 0.00075, 0.001, 0.0025, 0.005, 0.0075, 0.01]
#hyperparameters = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 15, 20, 30, 50, 100, 150] #number of neurons
nb_epochs, train_loss_list, validation_loss_list, train_accuracy_list, validation_accuracy_list, test_accuracy_list = [0.]*len(hyperparameters), [0.]*len(hyperparameters), [0.]*len(hyperparameters), [0.]*len(hyperparameters), [0.]*len(hyperparameters), [0.]*len(hyperparameters)
avg_train_accuracy, avg_validation_accuracy, avg_test_accuracy = [0.]*len(hyperparameters), [0.]*len(hyperparameters), [0.]*len(hyperparameters)
avg_nb_epochs = [0]*len(hyperparameters)
for k in range(n_avg):
    print(k)
    for i in range(len(hyperparameters)):
        w_list, b_list, nb_epochs[i], train_loss_list[i], validation_loss_list[i], train_accuracy_list[i], validation_accuracy_list[i] = train(X_train, y_train, [2, 50, 25, 1], batch_size=500, max_epochs=max_epochs, learning_rate=hyperparameters[i], acc_stop = 0.97, loss_diff=0.2, validation_split=0.2, mean=0, std_dev=0.05, optimizer='standard', verbose=False)
        z_list, a_list, pred = forward(X_test, w_list, b_list)
        test_accuracy_list[i] = accuracy(y_test, pred)
        avg_train_accuracy[i] += train_accuracy_list[i][-1]
        avg_validation_accuracy[i] += validation_accuracy_list[i][-1]
        avg_test_accuracy[i] += test_accuracy_list[i]        
        avg_nb_epochs[i] += nb_epochs[i]
        print(train_accuracy_list[i][-1], validation_accuracy_list[i][-1], test_accuracy_list[i])

avg_train_accuracy = list(map(lambda x: x/n_avg, avg_train_accuracy))
avg_validation_accuracy = list(map(lambda x: x/n_avg, avg_validation_accuracy))
avg_test_accuracy = list(map(lambda x: x/n_avg, avg_test_accuracy))
avg_nb_epochs = list(map(lambda x: x/n_avg, avg_nb_epochs))


In [ ]:
print(len(avg_train_accuracy))

In [ ]:
#Plot accuracies vs batch size
plt.figure(figsize=(10, 3))
plt.plot(hyperparameters, avg_train_accuracy, marker='x')
plt.plot(hyperparameters, avg_validation_accuracy, marker='o')
plt.plot(hyperparameters, avg_test_accuracy, marker='^')
plt.xscale('log')
plt.xticks(hyperparameters, labels=[str(x) for x in hyperparameters])
plt.title('Accuracy vs Learning rate')
plt.xlabel('Learning rate')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation', 'Test'], loc='upper right')
plt.grid(True)

In [ ]:
#Plot epoch for convergence
plt.figure(figsize=(10, 3))
plt.plot(hyperparameters, avg_nb_epochs, marker='o')
plt.xscale('log')
plt.xticks(hyperparameters, labels=[str(x) for x in hyperparameters])
plt.title('Convergence time vs Learning rate')
plt.xlabel('Learning rate')
plt.ylabel('Number of epochs')
plt.grid(True)

In [ ]:
plt.figure(figsize=(10, 3))
plt.subplot(1, 2, 1)
plt.plot(np.arange(1, nb_epochs + 1), train_loss_list)
plt.plot(np.arange(1, nb_epochs + 1), validation_loss_list)
plt.title('Loss vs epoch')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(np.arange(1, nb_epochs + 1), train_accuracy_list)
plt.plot(np.arange(1, nb_epochs + 1), validation_accuracy_list)
plt.title('Accuracy vs epoch')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='lower right')
plt.grid(True)
plt.show()

In [ ]:
z_list, a_list, pred = forward(X_test, w_list, b_list)

In [ ]:
test_accuracy = accuracy(y_test, pred)
print(f'accuracy: {test_accuracy*100}%')

In [ ]:
print(pred)

In [ ]:
def plot_prediction(X_test, pred):
    pattern_points = X_test.T
    points_c1 = [pattern_points[i] for i in range(len(pattern_points)) if pred[0][i] > 0.5]
    points_c2 = [pattern_points[i] for i in range(len(pattern_points)) if pred[0][i] <= 0.5]
    if points_c1 != []:
        x_c1, y_c1 = zip(*points_c1)
    if points_c2 != []:
        x_c2, y_c2 = zip(*points_c2)

    plt.figure(figsize=(6, 6))
    
    if points_c1 != []:
        plt.scatter(x_c1, y_c1, color='blue', s=1, label='C1', marker='o')
    if points_c2 != []:
        plt.scatter(x_c2, y_c2, color='red', s=1, label='C2')
   
    plt.title('Pattern Points')
    plt.xlabel('x1')
    plt.ylabel('x2')
    plt.grid(True)
    plt.legend()
    plt.xlim(-3, 3)
    plt.ylim(-3, 3)
    plt.text(-2.9, -2.9, "accuracy: "+str(test_accuracy), size=12, color=(0.7, 0, 0, 1))
    plt.show()

In [ ]:
def plot_prediction(X_test, pred):
    pattern_points = X_test.T
    x, y = zip(*pattern_points)

#    plt.figure(figsize=(6, 6))
    plt.scatter(x, y, s=0.25, c=pred[0], cmap=plt.cm.get_cmap("seismic"))
    plt.colorbar()       
    plt.title('Pattern Points')
    plt.xlabel('x1')
    plt.ylabel('x2')
    plt.grid(True)
    plt.xlim(-3, 3)
    plt.ylim(-3, 3)
    plt.text(-2.9, -2.9, "accuracy: "+str(test_accuracy), size=12, color=(0.7, 0, 0, 1))
    plt.show()

In [ ]:
plot_prediction(X_test, pred)

In [ ]:
def confusion_matrix(y, y_pred):
    assert y.shape[0] == 1
    assert y.shape == y_pred.shape
    
    # Round predictions to 0 or 1
    y_pred_rounded = np.round(y_pred)
    
    # Calculate true positives, true negatives, false positives, and false negatives
    TP = np.sum((y == 1) & (y_pred_rounded == 1))
    TN = np.sum((y == 0) & (y_pred_rounded == 0))
    FP = np.sum((y == 0) & (y_pred_rounded == 1))
    FN = np.sum((y == 1) & (y_pred_rounded == 0))

    conf_matrix = np.array([[TP, FP],
                            [FN, TN]])
    
    return conf_matrix

In [ ]:
import matplotlib

In [ ]:
def plot_confusion_matrix(conf_matrix):
    fig, ax = plt.subplots()

    cax = ax.matshow(conf_matrix, cmap=plt.cm.Blues, norm=matplotlib.colors.LogNorm())
    plt.colorbar(cax)

    ax.set_xticklabels([''] + ['Predicted C1', 'Predicted C2'])
    ax.set_yticklabels([''] + ['Actual C1', 'Actual C2'])

    for (i, j), val in np.ndenumerate(conf_matrix):
        ax.text(j, i, f'{val}', ha='center', va='center')

    plt.xlabel('Predicted labels')
    plt.ylabel('True labels')
    plt.title('Confusion Matrix')
    plt.show()


In [ ]:
conf_matrix = confusion_matrix(y_test, pred)

In [ ]:
plot_confusion_matrix(conf_matrix)